In [1]:
import numpy as np
from numpy import linalg as LA

M = []
M.append(np.array(np.mat('1 1; 0 1'))) # x
M.append(np.array(np.mat('1 0; 1 1'))) # y
M.append(np.array(np.mat('1 -1; 0 1'))) # x-1
M.append(np.array(np.mat('1 0; -1 1'))) # y-1
F = []
F.append(np.array(np.mat('1 2; 0 1'))) # x
F.append(np.array(np.mat('1 0; 2 1'))) # y
F.append(np.array(np.mat('1 -2; 0 1'))) # x-1
F.append(np.array(np.mat('1 0; -2 1'))) # y-1

# Devuelve la lista de digitos de num en base,
# y la completa con ceros a la dcha hasta que
# tenga longitud length
# A la dcha xq digits te devuelve la lista al reves
def extended_digits(num, base, length):
    dig = num.digits(base = 4)
    dig_len = len(dig)
    if dig_len != length:
        # Completamos con ceros a la dcha
        padding = [0]*(length - dig_len)
        dig = dig + padding
    return dig

# Manera menos elegante del mundo de hacer remove
# de x*x^-1 y de y*y^-1
def remove_sub2(word, sublist):
    for i in range(len(word)-1):
        if word[i:i+2] == sublist:
            word = word[:i] + word[i+2:]
    return word      

# Quita los x*x^-1, y*y^-1, x^-1*x, y^-1*y de la palabra 
def reduce_w(word):
    word = remove_sub2(word, [0,2])
    word = remove_sub2(word, [2,0])
    word = remove_sub2(word, [1,3])
    word = remove_sub2(word, [3,1])
    return word


#Encuentra la matriz -1 en palabras de longitud length
def finder(length, goal = np.array(np.mat('-1 0; 0 -1'))):
    results = []
    for i in srange(4^length):
        mult = np.array(np.mat('1 0; 0 1'))
        b4_digits = extended_digits(i, 4, length)
        for digit in b4_digits:
            mult = np.matmul(mult, M[digit])
        if np.array_equal(mult, goal):
            results.append(b4_digits)
    return results

# Esta cosa de aqui es útil porque, dada una ristra de palabras,
# Te devuelve las palabras que, al quitarle los x*x-1, no han quedado
# con la longitud size-antiguas (o sea, que no son repes)
def nuevas_de_verdad(words, size_antiguas):
    reduced = [reduce_w(word) for word in words]
    reduced_n = [] # Metemos aqui los -1 que no son de longitud 6
    for word in reduced:
        if len(word) != size_antiguas:
            reduced_n.append(word)
    return reduced_n

def produce(length, gens = F, num_gens = 4):
    results = []
    for i in srange(num_gens^length):
        mult = np.array(np.mat('1 0; 0 1'))
        b4_digits = extended_digits(i, num_gens, length)
        for digit in b4_digits:
            mult = np.matmul(mult, gens[digit])
        results.append(mult)
    return results



def not_in(elem, lista):
    for elem2 in lista:
        if elem[0][0] == elem2[0][0] and elem[0][1] == elem2[0][1] and elem[1][0] == elem2[1][0] and elem[1][1] == elem2[1][1]:
            return False
    return True

def elim_dups(lista):
    final = []
    for elem in lista:
        if not_in(elem, final):
            final.append(elem)
    return final

In [49]:
l = finder(6) #6 es la length minima donde aparece el -1
len(l), l

(16,
 [[3, 0, 0, 3, 0, 0],
  [0, 3, 0, 0, 3, 0],
  [3, 0, 3, 0, 3, 0],
  [3, 3, 0, 3, 3, 0],
  [2, 1, 1, 2, 1, 1],
  [1, 2, 1, 1, 2, 1],
  [2, 1, 2, 1, 2, 1],
  [2, 2, 1, 2, 2, 1],
  [1, 1, 2, 1, 1, 2],
  [1, 2, 1, 2, 1, 2],
  [2, 1, 2, 2, 1, 2],
  [1, 2, 2, 1, 2, 2],
  [0, 0, 3, 0, 0, 3],
  [0, 3, 0, 3, 0, 3],
  [3, 0, 3, 3, 0, 3],
  [0, 3, 3, 0, 3, 3]])

In [50]:
a8 = finder(8)   
a_reduced_8 = nuevas_de_verdad(a8, 6)

In [27]:
# Investigando saca de a_reduced_8 las palabras que son conjugadas de un generador o de su inverso:
# gen*algo*gen^-1
# Resulta que 'algo' es las cosas bonitas de tamanio 6 que son -1 (los elementos de l)
# Y que no es casualidad: gen*bonito*gen^-1 = gen*-1*gen^-1 = -1 xq -1 esta en el centro!

investigando = []
for word in a_reduced_8:
    if (word[0] == 0 and word[-1] == 2) or (word[0] == 2 and word[-1] == 0):
        investigando.append(word)
    if (word[0] == 1 and word[-1] == 3) or (word[0] == 3 and word[-1] == 1):
        investigando.append(word)
len(investigando), investigando  

(32,
 [[2, 3, 0, 0, 3, 0, 0, 0],
  [2, 3, 0, 3, 0, 3, 0, 0],
  [2, 3, 3, 0, 3, 3, 0, 0],
  [2, 2, 1, 1, 2, 1, 1, 0],
  [2, 1, 2, 1, 1, 2, 1, 0],
  [2, 2, 1, 2, 1, 2, 1, 0],
  [2, 2, 2, 1, 2, 2, 1, 0],
  [2, 3, 0, 3, 3, 0, 3, 0],
  [3, 3, 0, 0, 3, 0, 0, 1],
  [3, 0, 3, 0, 0, 3, 0, 1],
  [3, 3, 0, 3, 0, 3, 0, 1],
  [3, 3, 3, 0, 3, 3, 0, 1],
  [3, 2, 1, 1, 2, 1, 1, 1],
  [3, 2, 1, 2, 1, 2, 1, 1],
  [3, 2, 2, 1, 2, 2, 1, 1],
  [3, 2, 1, 2, 2, 1, 2, 1],
  [0, 1, 2, 1, 1, 2, 1, 2],
  [0, 1, 1, 2, 1, 1, 2, 2],
  [0, 1, 2, 1, 2, 1, 2, 2],
  [0, 1, 2, 2, 1, 2, 2, 2],
  [0, 0, 0, 3, 0, 0, 3, 2],
  [0, 0, 3, 0, 3, 0, 3, 2],
  [0, 3, 0, 3, 3, 0, 3, 2],
  [0, 0, 3, 3, 0, 3, 3, 2],
  [1, 0, 3, 0, 0, 3, 0, 3],
  [1, 1, 1, 2, 1, 1, 2, 3],
  [1, 1, 2, 1, 2, 1, 2, 3],
  [1, 2, 1, 2, 2, 1, 2, 3],
  [1, 1, 2, 2, 1, 2, 2, 3],
  [1, 0, 0, 3, 0, 0, 3, 3],
  [1, 0, 3, 0, 3, 0, 3, 3],
  [1, 0, 3, 3, 0, 3, 3, 3]])

In [46]:
# Aqui meto todas las demas (las que no son conjugadas)
lasquequedan = [word for word in a_reduced_8 if word not in investigando]
len(lasquequedan)

32

In [47]:
# De las que quedan, meto aqui las que empiezan y acaban por el mismo generador 
# (Son una cosa, y la misma cosa dada la vuelta!!)
[word for word in lasquequedan if word[0] == word[-1]]

[[0, 3, 0, 1, 0, 3, 0, 0],
 [0, 0, 3, 0, 1, 0, 3, 0],
 [1, 2, 1, 0, 1, 2, 1, 1],
 [1, 1, 2, 1, 0, 1, 2, 1],
 [2, 2, 1, 2, 3, 2, 1, 2],
 [2, 1, 2, 3, 2, 1, 2, 2],
 [3, 3, 0, 3, 2, 3, 0, 3],
 [3, 0, 3, 2, 3, 0, 3, 3]]

In [44]:
# Y aqui todo lo demas
# Hay algunas que estan dadas la vuelta, y otras que mantienen los endpoints e invierten el orden de los 3 pares internos
# Veanse los 2 primeros elementos
#      0  00 30 10  3
#      0  10 30 00  3
sorted([word1 for word1 in lasquequedan if word1 not in [word for word in lasquequedan if word[0] == word[-1]]])        

[[0, 0, 0, 3, 0, 1, 0, 3],
 [0, 1, 0, 3, 0, 0, 0, 3],
 [0, 1, 2, 1, 1, 1, 2, 1],
 [0, 3, 0, 0, 0, 3, 0, 1],
 [0, 3, 2, 3, 0, 3, 3, 3],
 [0, 3, 3, 3, 0, 3, 2, 3],
 [1, 0, 1, 2, 1, 1, 1, 2],
 [1, 0, 3, 0, 0, 0, 3, 0],
 [1, 1, 1, 2, 1, 0, 1, 2],
 [1, 2, 1, 1, 1, 2, 1, 0],
 [1, 2, 2, 2, 1, 2, 3, 2],
 [1, 2, 3, 2, 1, 2, 2, 2],
 [2, 1, 0, 1, 2, 1, 1, 1],
 [2, 1, 1, 1, 2, 1, 0, 1],
 [2, 1, 2, 2, 2, 1, 2, 3],
 [2, 2, 2, 1, 2, 3, 2, 1],
 [2, 3, 0, 3, 3, 3, 0, 3],
 [2, 3, 2, 1, 2, 2, 2, 1],
 [3, 0, 0, 0, 3, 0, 1, 0],
 [3, 0, 1, 0, 3, 0, 0, 0],
 [3, 0, 3, 3, 3, 0, 3, 2],
 [3, 2, 1, 2, 2, 2, 1, 2],
 [3, 2, 3, 0, 3, 3, 3, 0],
 [3, 3, 3, 0, 3, 2, 3, 0]]

In [52]:
a10 = finder(10)

In [57]:
a_reduced_10 = nuevas_de_verdad(a10, 6)
len(a_reduced_10)

2712

In [10]:
a11 = finder(11)

In [58]:
a_reduced_11 = nuevas_de_verdad(a11, 6)
len(a_reduced_11), a_reduced_11

(0, [])

In [13]:
a12 = finder(12)

In [60]:
a_reduced_12 = nuevas_de_verdad(a12, 6)
len(a_reduced_12)

66880

In [7]:

goals = []
goals.append(np.array(np.mat('1 -1; 1 0')))
goals.append(np.array(np.mat('1 1; -1 0')))
goals.append(np.array(np.mat('2 -1; 1 0')))
goals.append(np.array(np.mat('2 1; -1 0')))
goals.append(np.array(np.mat('0 1;-1  0')))
goals.append(np.array(np.mat('0 -1;1  0')))
for goal in goals:
    res = []
    length = 2
    while not len(res):
        res = finder(length, goal)
        if len(res):
            word = ['x' if i == 0 else 'y' if i == 1 else 'x^-1' if i == 2 else 'y^-1' for i in res[0]]
            print goal, word
        length += 1

[[ 1 -1]
 [ 1  0]] ['y', 'x^-1']
[[ 1  1]
 [-1  0]] ['y^-1', 'x']
[[ 2 -1]
 [ 1  0]] ['x', 'y', 'x^-1']
[[ 2  1]
 [-1  0]] ['x^-1', 'y^-1', 'x']
[[ 0  1]
 [-1  0]] ['x', 'y^-1', 'x']
[[ 0 -1]
 [ 1  0]] ['y', 'x^-1', 'y']


In [2]:
res = []
for i in range(10):
    res.append(produce(i))

In [3]:
for result in res:
    for mat in result:
        
        #print mat[0], '\n', mat[1], '\n'

IndentationError: expected an indented block (<ipython-input-3-c83095986e07>, line 4)

In [18]:
len(res1), len(res2)

(1365, 1398101)

In [2]:
res1 = []
res2 = []
for i in range(7):
    aux = produce(i)
    for mat in aux:
        res1.append(mat)
for i in range(10):
    for mat in aux:
        res2.append(mat)


In [6]:
aux = produce(i, gens = M)

not_free = []
for mat in res2:
    if not_in(mat, res1):
        not_free.append(mat)

In [3]:
res1 = elim_dups(res1)

contraejs = []
for mat in res2:
    if is_odd(mat[0][0]) and is_odd(mat[1][1]) and is_even(mat[1][0]) and is_even(mat[0][1]) and not_in(mat, res1):
        contraejs.append(mat)

print len(contraejs)
contraejs = elim_dups(contraejs)
print len(contraejs)

9424
253


In [ ]:
# Aniadimos las palabras de longitud 11
aux = produce(11, gens = M)


In [ ]:

for mat in aux:
    if is_odd(mat[0][0]) and is_odd(mat[1][1]) and is_even(mat[1][0]) and is_even(mat[0][1]) and not_in(mat, res1) and not_in(mat, contraejs):
        contraejs.append(mat)
    if not_in(mat , res2):
        res2.append(mat)


In [4]:
for mat in contraejs[:10]:
    print mat[0], '\n', mat[1], '\n'

[3 4] 
[2 3] 

[-1 -2] 
[2 3] 

[-1  0] 
[-2 -1] 

[3 2] 
[-2 -1] 

[3 2] 
[4 3] 

[-1 -2] 
[ 0 -1] 

[ 3 -2] 
[ 2 -1] 

[-1  0] 
[ 2 -1] 

[-1  2] 
[-2  3] 

[ 3 -4] 
[-2  3] 



In [7]:
print len(res1)
for mat in res1[:10]:
    w, v = LA.eig(mat)
    print mat[0], '\n', mat[1], '\n'

485
[1 0] 
[0 1] 

[1 2] 
[0 1] 

[1 0] 
[2 1] 

[ 1 -2] 
[0 1] 

[1 0] 
[-2  1] 

[1 4] 
[0 1] 

[1 2] 
[2 5] 

[1 2] 
[-2 -3] 

[5 2] 
[2 1] 

[1 0] 
[4 1] 



In [4]:
def mod(lst, modulo):
    return [elem%modulo for elem in lst]

f = open('matrices4', 'w')
string = ''
for mat in res1:
    #string += str(mat[0]) + '\t\t\t' + str(mod(mat[0], 3)) + '\t\t\t' + str(mod(mat[0], 4)) + '\t\t\t' + str(mod(mat[0], 5)) + '\n'
    #string += str(mat[1]) + '\t\t\t' + str(mod(mat[1], 3)) + '\t\t\t' + str(mod(mat[1], 4)) + '\t\t\t' + str(mod(mat[1], 5)) + '\n'
    if 3 in mod(mat[0], 4) or 3 in mod(mat[1], 4):
        print 'Eres idiota'
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# Parece q si eres libre, no puedes ser 3 modulo 4!

f = open('contraejs4', 'w')
string = ''
for mat in contraejs:
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# No es suficiente.
# Lloro un pelin na mas
# A lo mejor si q es suficiente! 
# No entiendo muy bien xq, pero mirando libres de longitud 6, pa mi q estaba
# mirando matrices de longitud 12 y por tanto ya habia cubierto las de 
# longitud 10, pero parece q no.
# Las normales de longitud 10 que no tenian ningun 3 en su modulo y no estaban
# dentro de las libres de longitud 6, si estan dentro de las libres
# de longitud 7. wtf

In [16]:
def mod(lst, modulo):
    return [elem%modulo for elem in lst]
def mi_forma_bonita(mat):
    boolean = (mat[0][0]%4 == 1) and (mat[1][1]%4 == 1)
    boolean = boolean and (mat[0][1]%2 == 0) and (mat[1][0]%2 == 0)
    return boolean

f = open('libres_mod_4', 'w')
string = ''
for mat in res1:
    #string += str(mat[0]) + '\t\t\t' + str(mod(mat[0], 3)) + '\t\t\t' + str(mod(mat[0], 4)) + '\t\t\t' + str(mod(mat[0], 5)) + '\n'
    #string += str(mat[1]) + '\t\t\t' + str(mod(mat[1], 3)) + '\t\t\t' + str(mod(mat[1], 4)) + '\t\t\t' + str(mod(mat[1], 5)) + '\n'
    if mi_forma_bonita(mat) == False:
        print 'Eres idiota2'
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# Parece q si eres libre, no puedes ser 3 modulo 4!

f = open('no_libres_mod_4', 'w')
string = ''
for mat in not_free:
    if mi_forma_bonita(mat) == True:
        print 'Eres idiota'
    string += str(mod(mat[0], 4)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 4)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

# No es suficiente.
# Lloro un pelin na mas
# A lo mejor si q es suficiente! 
# No entiendo muy bien xq, pero mirando libres de longitud 6, pa mi q estaba
# mirando matrices de longitud 12 y por tanto ya habia cubierto las de 
# longitud 10, pero parece q no.
# Las normales de longitud 10 que no tenian ningun 3 en su modulo y no estaban
# dentro de las libres de longitud 6, si estan dentro de las libres
# de longitud 7. wtf

In [39]:
# Si con modulos 2 y 4 van bien, a lo mejor el 8 sirve de algo, pero yo q se.
# Desde luego, las libres me van a salir no son ni 3 ni 7, ¿pero las otras?

f = open('matrices8', 'w')
string = ''
for mat in res1:
    string += str(mod(mat[0], 8)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 8)) + '\t\t\t' + str(mat[1]) + '\n\n'

f.write(string)
f.close()

f = open('contraejs8', 'w')
string = ''
for mat in contraejs:
    string += str(mod(mat[0], 8)) + '\t\t\t' + str(mat[0]) + '\n'
    string += str(mod(mat[1], 8)) + '\t\t\t' + str(mat[1]) + '\n\n'
        
f.write(string)
f.close()

# Pues no, no sirve de na D: